In [1]:
import pandas as pd

In [2]:
db = pd.read_excel('fitness.xlsx')

In [3]:
db.head()

,вопрос,группа,target
0,Последнее время даже в обычных супермаркетах с...,другое,1
1,Последнее время даже в обычных супермаркетах с...,питание,0
2,Последнее время даже в обычных супермаркетах с...,тренажерный зал,0
3,"Что лучше посещать – тренажерные залы, бассейн...",тренажерный зал,1
4,"Что лучше посещать – тренажерные залы, бассейн...",питание,0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('russian')
noise = noise+['это', 'тыс', 'млн', 'млрд', 'также']
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш','ваш','очень', 'мочь','этот', 'обычно','даже',
                'прошлый', 'раз','добрый','здравствуйте', 'здравствовать', 'подскажите', 'покажите',
                'какой',
                'такой', ' ', '\n']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\volko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def dataprep(inf):
    dp = [words_only(x.lower()) for x in inf]
    dp = [x.replace(' ',',').split(',') for x in dp]
    long_words = []
    for j in tqdm(dp):
        long_words.append([x for x in j if len(x)>1])
    latterm_wo_stopw = []
    for k in tqdm(long_words):
        latterm_wo_stopw.append(' '.join([x for x in k if x not in noise]))
    return latterm_wo_stopw

In [8]:
from rnnmorph.predictor import RNNMorphPredictor
predictor=RNNMorphPredictor(language='ru')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
def lem(a):
    res = []
    for i in tqdm(a):
        i = i.split(' ')
        forms = predictor.predict(i)
        res.append([word.normal_form for word in forms])
    return res

In [10]:
def lem_wo_stop(a):
    latterm = []
    for i in tqdm(a):
        latterm.append(' '.join([x for x in i if x not in mystoplemmas]))
    return latterm

In [11]:
from tqdm import tqdm

In [12]:
data = dataprep(db['вопрос'])

100%|█████████████████████████████████████████████████████████████████████████████| 852/852 [00:00<00:00, 44961.02it/s]


In [13]:
data = lem(data)

100%|████████████████████████████████████████████████████████████████████████████████| 852/852 [00:10<00:00, 58.02it/s]


In [14]:
data = lem_wo_stop(data)

100%|████████████████████████████████████████████████████████████████████████████| 852/852 [00:00<00:00, 213499.04it/s]


In [15]:
tfidf = TfidfVectorizer(ngram_range=(1,1))

In [16]:
tfidf_data = tfidf.fit_transform(data)

In [17]:
data[0]

'последний время обычный супермаркет стать продавать энергетический напиток сын год буквально жить видеть напиток бар вредный употреблять'

In [18]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [19]:
from gensim.models import Word2Vec
import time
import logging #для отображения состояния
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt = '%H:%M:%S', level = logging.INFO)

C:\Users\volko\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [20]:
groups=db['группа'].drop_duplicates().values

In [21]:
groups_data=db[db['target']==1]

In [22]:
groups_dict={}
for i in groups:
    tab=groups_data[groups_data['группа']==i]
    z=' '.join(tab['вопрос'])
    groups_dict[i]=z

In [23]:
list(groups_dict.values())

['Последнее время даже в обычных супермаркетах стали продавать энергетические напитки. Мой сын, ему 17 лет, буквально не может без них жить. Я видела, что такие напитки есть у Вас в баре. Не вредно ли их употреблять? Какая температура должна быть в бане? Как часто можно посещать баню? Что такое фитнес - бар? Почему в основном зале (да и вообще) стало так невероятно скользко? С обувью все нормально. Хоть бы положили возле смита и стойки противоскользящие куски покрытия, невозможно же Добрый день , скажите пожалуйста , есть ли стол для Армрестлинга в клубе на Беляево ??? Большое спасибо за ответ ) Подскажите пожалуйста пороль от Wifi Можно ли привести друга и он 1 раз бесплатно сходит?(у него нету гражданства РФ) Здравствуйте , можно ли восстановить потерянную карту ? Здравствуйте! В период отключения горячей воды в зале будет горячее водоснабжение, предусмотрены ли водонагреватели? У вас можно расплатиться картой?',
 'Если тренироваться, но совершенно не следить за своим питанием, дости

In [24]:
data_w2v_1 = dataprep(list(groups_dict.values()))
data_w2v_1 = lem(data_w2v_1)
data_w2v_1 = lem_wo_stop(data_w2v_1)

100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 4346.08it/s]


In [25]:
data_w2v=[x.split() for x in data_w2v_1]

In [26]:
model = Word2Vec(min_count = 1,
                     window = 15,
                     size = 100,
                     alpha = 0.03, 
                     min_alpha = 0.0007,
                     workers = cores-1)

In [27]:
%%time
model.build_vocab(sentences = data_w2v, 
                  progress_per = 10000)

model.train(sentences = data_w2v,
            total_examples = len(data), 
            epochs = 50, 
            report_delay = 1)

INFO - 11:55:33: collecting all words and their counts
INFO - 11:55:33: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:55:33: collected 652 word types from a corpus of 1807 raw words and 13 sentences
INFO - 11:55:33: Loading a fresh vocabulary
INFO - 11:55:33: effective_min_count=1 retains 652 unique words (100% of original 652, drops 0)
INFO - 11:55:33: effective_min_count=1 leaves 1807 word corpus (100% of original 1807, drops 0)
INFO - 11:55:33: deleting the raw counts dictionary of 652 items
INFO - 11:55:33: sample=0.001 downsamples 61 most-common words
INFO - 11:55:33: downsampling leaves estimated 1274 word corpus (70.5% of prior 1807)
INFO - 11:55:33: estimated required memory for 652 words and 100 dimensions: 847600 bytes
INFO - 11:55:33: resetting layer weights
INFO - 11:55:33: training model with 3 workers on 652 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=15
INFO - 11:55:33: worker thread finished; awaiting finish 

INFO - 11:55:34: EPOCH - 17 : training on 1807 raw words (1284 effective words) took 0.0s, 164031 effective words/s
WARNING - 11:55:34: EPOCH - 17 : supplied example count (13) did not equal expected count (852)
INFO - 11:55:34: worker thread finished; awaiting finish of 2 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 1 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 0 more threads
INFO - 11:55:34: EPOCH - 18 : training on 1807 raw words (1286 effective words) took 0.0s, 153721 effective words/s
WARNING - 11:55:34: EPOCH - 18 : supplied example count (13) did not equal expected count (852)
INFO - 11:55:34: worker thread finished; awaiting finish of 2 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 1 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 0 more threads
INFO - 11:55:34: EPOCH - 19 : training on 1807 raw words (1260 effective words) took 0.0s, 119519 effective words/s
WARNING - 

INFO - 11:55:34: worker thread finished; awaiting finish of 1 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 0 more threads
INFO - 11:55:34: EPOCH - 36 : training on 1807 raw words (1251 effective words) took 0.0s, 119981 effective words/s
WARNING - 11:55:34: EPOCH - 36 : supplied example count (13) did not equal expected count (852)
INFO - 11:55:34: worker thread finished; awaiting finish of 2 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 1 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 0 more threads
INFO - 11:55:34: EPOCH - 37 : training on 1807 raw words (1278 effective words) took 0.0s, 159551 effective words/s
WARNING - 11:55:34: EPOCH - 37 : supplied example count (13) did not equal expected count (852)
INFO - 11:55:34: worker thread finished; awaiting finish of 2 more threads
INFO - 11:55:34: worker thread finished; awaiting finish of 1 more threads
INFO - 11:55:34: worker thread finished; awaiting f

Wall time: 1.21 s


In [28]:
model.most_similar('карта')

C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
INFO - 11:55:34: precomputing L2-norms of word weight vectors
C:\Users\volko\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('студент', 0.9997413158416748),
 ('спарта', 0.9994149804115295),
 ('скидка', 0.999409556388855),
 ('кинуть', 0.999191403388977),
 ('ночь', 0.9991479516029358),
 ('заморозить', 0.9991403818130493),
 ('лс', 0.9990387558937073),
 ('купить', 0.9989029169082642),
 ('покупать', 0.9988566637039185),
 ('откуда', 0.9988155364990234)]

In [29]:
tfidf_data_group = tfidf.transform(data_w2v_1)

In [30]:
import numpy as np

In [31]:
word_matrix = []
for j in tqdm(range(len(tfidf.get_feature_names()))):
    try:
        res = model[tfidf.get_feature_names()[j]]
    except:
        res = np.zeros(100)
    word_matrix.append(res)

  0%|                                                                                          | 0/649 [00:00<?, ?it/s]C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████████████████████████████████████████████████████████████████████████| 649/649 [00:00<00:00, 2400.63it/s]


In [32]:
tags_phrase = np.dot(tfidf_data.toarray(),np.array(word_matrix))

In [33]:
tags_phrase_1 = np.dot(tfidf_data_group.toarray(),np.array(word_matrix))

In [34]:
tags_phrase_1[0]

array([ 0.35633098,  1.33743277,  2.17991215,  2.59380993,  2.09451235,
        5.16576518,  0.20954788, -1.19376109,  1.00264923,  1.94208401,
        1.30347299, -4.1484054 ,  1.51092952,  0.79107333, -3.24178579,
        0.31525795, -2.19347449, -2.57969566,  0.77666127, -0.2145404 ,
       -1.78607033,  0.37466831, -0.03988728,  1.33006832,  2.08966158,
        1.70446126, -1.8889795 ,  2.14187387,  3.11592934,  1.35410866,
       -0.26860093,  1.46688445,  0.27738229,  0.08301827,  2.21072015,
        0.73822328, -4.46713785,  2.32355479,  0.05254393,  0.37969669,
       -0.86993528,  4.2929213 ,  0.52591502, -1.0947955 , -2.16402116,
       -0.08286078, -1.94892747, -0.53470223, -0.80482836,  2.50302101,
       -0.63515226, -1.3966966 ,  2.09234105, -3.83961516, -2.27988733,
        3.7617847 ,  1.19669545, -2.33157882, -1.29202581, -0.82956803,
       -0.68311843, -1.87579351, -0.01820362,  1.7623045 , -2.92289131,
       -0.95362474, -1.0408066 ,  0.99275187,  0.52023233, -1.61

In [35]:
for i in range(len(groups_dict.keys())):
    groups_dict[list(groups_dict.keys())[i]]=tags_phrase_1[i]

In [38]:
groups_data=[]
for i in db['группа']:
    groups_data.append(groups_dict[i])

In [41]:
factor_tag = [list(tags_phrase.tolist()[i]) + list(groups_data[i]) for i in range(len(tags_phrase))]

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
y=db['target']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(factor_tag, y, stratify=y, test_size=0.2, random_state=42)

In [47]:
from  sklearn.linear_model import LogisticRegression

In [48]:
model_logistic=LogisticRegression()

In [49]:
model_logistic.fit(X_train, y_train)

C:\Users\volko\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [50]:
from sklearn.metrics import *

In [51]:
y_pred=model_logistic.predict(X_test)
accuracy_score(y_pred, y_test)

0.6432748538011696

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [78]:
model_forest=RandomForestClassifier(n_estimators=200)

In [79]:
model_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [80]:
y_pred=model_forest.predict(X_test)
accuracy_score(y_pred, y_test)

0.7426900584795322

In [59]:
import xgboost as xgb

In [72]:
xgb_model = xgb.XGBClassifier(n_estimators=200)

In [73]:
xgb_model.fit(np.array(X_train), y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [74]:
y_pred=xgb_model.predict(np.array(X_test))
accuracy_score(y_pred, y_test)

0.7076023391812866